In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import pymysql as pydb
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine, text, Column, Integer, String, DateTime
from sqlalchemy.orm import sessionmaker, declarative_base
from IPython.display import display, clear_output

import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors
from scipy.stats import norm

import warnings
import matplotlib.ticker as ticker
warnings.filterwarnings("ignore")

### Load Raw Data

In [ ]:
result_dataframe = pd.read_csv('report_problem_raw.csv')

result_dataframe = result_dataframe[result_dataframe['problem'].str.contains('คืนงาน')]

result_dataframe

,report_date,report_month,problem,order_id,report_datetime,day_of_week,selected_professional_id,cancel_professional_id,booking_type,remark
0,2025-01-01,2025-01,คืนงาน (กะทันหัน),1192844,2025-01-01 06:55:00,Wed,0,8246,งานใครรับก็ได้,คุณแม่บ้านแจ้งเป็นไข้ทับระดูและท้องเสีย ลุกไม...
1,2025-01-01,2025-01,คืนงาน (กะทันหัน),1191526,2025-01-01 11:03:00,Wed,0,10287,งานใครรับก็ได้,คุณแม่บ้านแจ้งคืนงานเนื่องจาก ไม่สบาย มีไข้ น้...
2,2025-01-01,2025-01,คืนงาน (ล่วงหน้า),1154597,2025-01-01 10:11:00,Wed,0,79,งานใครรับก็ได้,คุณแม่บ้านแจ้งคืนงานเนื่องจาก แม่บ้านประสบอุบั...
3,2025-01-01,2025-01,คืนงาน (กะทันหัน),1018893,2025-01-01 11:50:00,Wed,0,11090,งานใครรับก็ได้,คุณแม่บ้านแจ้งคืนงาน 14 งาน เนื่องจากจะลาออก ต...
4,2025-01-01,2025-01,คืนงาน (กะทันหัน),1108624,2025-01-01 14:45:00,Wed,0,7810,งานใครรับก็ได้,คุณแม่บ้านแจ้งมีอาการปวดหัวจึงยังไม่สะดวกเข้าบ...
...,...,...,...,...,...,...,...,...,...,...
2177,2025-06-23,2025-06,คืนงาน (กะทันหัน),1373965,2025-06-23 13:11:00,Mon,0,12857,งานใครรับก็ได้,รับเคสห้องโอเปอ คุณแม่บ้านแจ้งว่ารู้สึกเหมือนจ...
2178,2025-06-23,2025-06,คืนงาน (กะทันหัน),1369518,2025-06-23 15:48:00,Mon,11148,11148,งานจองตรง,แม่บ้านแจ้งว่ารถล้ม ไม่สามารถเข้าให้บริการได้ค่ะ
2179,2025-06-23,2025-06,คืนงาน (โครงการป้อนงานคุณแม่บ้าน),1374291,2025-06-23 16:22:00,Mon,0,7065,งาน Auto-Assign,คุณแม่บ้านคืนงานเนื่องจากไม่สะดวกเข้าให้บริการ...
2180,2025-06-23,2025-06,คืนงาน (กะทันหัน),1373881,2025-06-23 17:13:00,Mon,0,12922,งานใครรับก็ได้,คุณแม่บ้านติดต่อเข้ามาที่ CC แจ้งคืนงาน เนื่อง...


In [86]:
# ========================================
# FIXED THAI CLASSIFIER V3 - ADDRESSES ALL WRONG CLASSIFICATIONS
# ========================================

import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

# Check ThaiNLP availability
try:
    from pythainlp import word_tokenize, normalize
    from pythainlp.corpus import thai_stopwords
    THAINLP_AVAILABLE = True
    print("✅ ThaiNLP is available!")
except ImportError:
    THAINLP_AVAILABLE = False
    print("⚠️ ThaiNLP not available. Using basic preprocessing.")

class FixedThaiClassifierV3:
    """FIXED Thai text classifier V3 - addresses all wrong classifications"""
    
    def __init__(self):
        print("🚀 Initializing FIXED Thai Classifier V3...")
        
        # Your exact 17 categories
        self.categories = [
            'ติดธุระด่วน', 'ไม่สะดวกเดินทาง', 'คุณลูกค้าไม่สะดวกเลื่อนวัน-เวลา', 
            'ดูวันผิด', 'ไม่สะดวกเข้าให้บริการ(เงื่อนไขคุณลูกค้า)', 'ป่วย', 
            'รับงานอื่น', 'ปัญหาสุขภาพ', 'ระยะทาง', 'ติดงานประจำ', 
            'กลับต่างจังหวัด', 'ดูแลครอบครัว', 'อุปกรณ์ไม่พร้อม', 
            'แจ้งลาออก', 'อุบัติเหตุ', 'ที่อยู่ไม่ตรงในใบงาน', 'สัตว์เลี้ยง'
        ]
        
        # COMPLETELY REBUILT keyword patterns - fixing all wrong classifications
        self.keyword_patterns = {
            'ติดธุระด่วน': {
                'critical': ['ธุระด่วน', 'มีเหตุสุดวิสัย', 'เหตุการณ์ฉุกเฉิน', 'ฉุกเฉิน', 'เป็นวันไหว้', 'ไปศาล', 'เช้งเม้ง'],
                'high': ['ติดธุระ', 'มีธุระ', 'มีเหตุจำเป็น', 'ความจำเป็น', 'จำเป็น', 'ภาระกิจ', 'ติดภาระกิจ', 'ต่างประเทศ', 'มอบตัว', 'เข้าค่าย'],
                'medium': ['ธุระ', 'ด่วน', 'เร่งด่วน', 'กะทันหัน', 'นัดหมอ', 'เสียชีวิต', 'ขาวดำ'],
                'low': ['งานด่วน', 'ธุระส่วนตัว', 'ต้องไปด่วน']
            },
            
            'ไม่สะดวกเดินทาง': {
                'critical': ['รถเสีย', 'รถชำรุด', 'รถพัง', 'ไม่มีรถ', 'ยางแตก', 'รถไม่สตาร์ท'],
                'high': ['ไม่สะดวกเดินทาง', 'เดินทางลำบาก', 'ไม่สามารถเดินทาง', 'ไม่ชินเส้นทาง', 'ไปไม่ถูก', 'ไปไม่ทัน'],
                'medium': ['รถติด', 'ไปไม่ได้', 'รถประจำทาง', 'ขนส่ง'],
                'low': ['เดินทาง', 'รถ', 'ไม่สะดวก', 'ฝน', 'ถนน']
            },
            
            'คุณลูกค้าไม่สะดวกเลื่อนวัน-เวลา': {
                'critical': ['ลูกค้าเลื่อนนัด', 'ลูกค้ายกเลิกนัด', 'เลื่อนการจอง', 'ลูกค้าขอเลื่อนวันเวลานัดหมายบ่อยครั้ง'],
                'high': ['ลูกค้าไม่สะดวกเลื่อน', 'ว่างไม่ตรงกัน', 'ขอเลื่อนวันเวลา'],
                'medium': ['เลื่อน', 'นัดหมาย', 'ไม่สะดวกเลื่อน', 'ขอเลื่อนงาน'],
                'low': ['เลื่อนเวลา', 'เลื่อนวัน', 'เปลี่ยนเวลา', 'ลูกค้า']
            },
            
            'ดูวันผิด': {
                'critical': ['ดูวันผิด', 'ดูเวลาผิด', 'อ่านวันผิด', 'เข้าใจผิดวัน', 'เผลอกดรับงานผิด', 'ดูวันที่ผิด'],
                'high': ['กดรับงานผิด', 'ดูวันที่ผิด', 'ดูสถานที่ผิด', 'กดรับโดยไม่ตั้งใจ', 'ลืมค่ะ', 'ไม่ได้ดู'],
                'medium': ['ดูผิด', 'กดผิด', 'สับสน', 'ผิดพลาด', 'เผลอกด'],
                'low': ['อ่าน', 'เข้าใจผิด', 'นึกว่า', 'กด']
            },
            
            'ไม่สะดวกเข้าให้บริการ(เงื่อนไขคุณลูกค้า)': {
                'critical': ['ไม่สะดวกเข้าให้บริการ', 'กำจัดกลิ่น', 'ลูกค้าเรื่องเยอะ', 'รายละเอียดเยอะ', 'จำนวนชั่วโมงไม่เพียงพอ', 'งานไม่บาลานซ์กัน', 'ผ้าลูกค้ารีดค่อนข้างยาก', 'สื่อสารกับลูกค้าไม่เข้าใจ', 'ลูกค้าให้เลิกทำงาน', 'ทำความสะอาดมีกลิ่นไม่สะอาด'],
                'high': ['ไม่เข้าเงื่อนไข', 'นอกเหนือขอบเขต', 'ต่างชาติ', 'เวลาไม่เพียงพอ', 'ชั่วโมงไม่พอ', 'เรื่องเยอะ', 'ไม่สะดวกใจจะเข้าบริการ', 'เสียโอกาสในการรับงานอื่น', 'รู้สึกอึดอัด', 'คำพูดของลูกค้า', 'ไม่สบายใจ', 'ไม่ให้ต่อเวลา', 'ชักชวนคุณแม่บ้าน'],
                'medium': ['เงื่อนไข', 'ขอบเขต', 'รายละเอียด', 'ค่าจอดรถ', 'ไม่สะดวกรับ', 'ไม่เหมาะกับแม่บ้าน', 'เลื่อนบ่อยครั้ง', 'เลื่อนกะทันหัน'],
                'low': ['ไม่ชอบกลิ่น', 'เปลี่ยน', 'พฤติกรรม', 'ไม่พอใจ', 'ไม่ถนัด']
            },
            
            'ป่วย': {
                'critical': ['ป่วย', 'ไข้', 'ไมเกรน', 'ความดันสูง', 'ไปหาหมอ', 'อาหารเป็นพิษ', 'ไปโรงพยาบาล', 'ข้อเท้าอักเสบ', 'ไซนัสอักเสบ', 'ผ่าฟันคุด', 'ท้องเสีย', 'งูสวัด', 'ปวดฟันมานาน', 'แขนอักเสบ', 'กล้ามเนื้ออักเสบ', 'ปวดมือ', 'มือมีอาการบวม', 'โดนเหล็กฝาท่อบาดมือ', 'กลัวติดเชื้อ'],
                'high': ['เจ็บป่วย', 'เป็นไข้', 'รีบไปหาหมอ', 'ต้องไปหาหมอ', 'เจ็บป่วยเฉียบพลัน', 'ต้องเข้ารับผ่าตัด', 'หมอสั่งให้พัก'],
                'medium': ['ปวดหัว', 'ปวดท้อง', 'หวัด', 'ไอ', 'เจ็บคอ', 'ปวดฟัน', 'บาดเจ็บ', 'บวมเจ็บ', 'ไม่สบายตัว', 'เวียนหัว', 'ไม่สบาย'],
                'low': ['หมอ', 'โรงพยาบาล', 'รักษา', 'ยา', 'อาการ']
            },
            
            'รับงานอื่น': {
                'critical': ['รับงานอื่น', 'ไปรับงานอื่น', 'จะไปรับงาน', 'ได้งานประจำ'],
                'high': ['งานอื่น', 'มีงานอื่น', 'ติดงานอื่น', 'งานที่ดีกว่า', 'งานด่วนเข้ามา'],
                'medium': ['งานใหม่', 'งานเพิ่ม', 'ทำงานอื่น'],
                'low': ['งานทับ', 'เวลาชน', 'ตารางชน']
            },
            
            'ปัญหาสุขภาพ': {
                'critical': ['ปัญหาสุขภาพ', 'สุขภาพไม่ค่อยดี', 'สุขภาพไม่เอื้อ', 'เริ่มทำไม่ไหว้แล้ว', 'ปัญหาทางด้านสุขภาพ'],
                'high': ['ครรภ์', 'ตั้งครรภ์', 'อายุมาก', 'ผู้สูงอายุ', 'ทำงานหนักไม่ได้', 'ร่างกายไม่อำนวย', 'สภาพร่างกายไม่ไหว', 'เข้าให้บริการไม่ไหว', 'ตั้งครรภ์', 'ปวดเข่า', 'ปวดขา', 'ปวดแขน'],
                'medium': ['สุขภาพไม่ดี', 'ไม่แข็งแรง', 'โรคประจำตัว', 'ร่างกายอ่อนแอ'],
                'low': ['สุขภาพ', 'ข้อเสื่อม', 'พิการ', 'ชรา']
            },
            
            'ระยะทาง': {
                'critical': ['ระยะทางไกล', 'ไกลมาก', 'ไกลเกินไป', 'ไม่คุ้มค่า', 'ไกลไม่สะดวกเดินทาง', 'นอกเขต'],
                'high': ['ค่าเดินทางแพง', 'ระยะทางค่อนข้างไกล', 'ไกลจากที่พักมาก'],
                'medium': ['ระยะทาง', 'ไกล', 'เดินทางไกล', 'ใช้เวลานาน'],
                'low': ['เดินทางเหนื่อย', 'อยู่ต่างเขต']
            },
            
            'ติดงานประจำ': {
                'critical': ['ติดงานประจำ', 'ติดงานออฟฟิศ', 'ติดที่ทำงาน', 'ติดงานขนส่ง'],
                'high': ['ติดประชุม', 'มีประชุม', 'ติดงานจำเป็น', 'พึ่งได้งานประจำ'],
                'medium': ['งานประจำ', 'ที่ทำงาน', 'บริษัท', 'ประชุม', 'สัมมนา'],
                'low': ['สำนักงาน', 'เวลาทำงาน']
            },
            
            'กลับต่างจังหวัด': {
                'critical': ['กลับต่างจังหวัด', 'ไปต่างจังหวัด', 'กลับบ้านเกิด', 'ติดธุระต่างจังหวัด', 'ทำธุระต่างจังหวัด', 'กลับเชียงใหม่', 'กลับขอนแก่น', 'ไปเผาศพคุณตา ที่ตจว', 'เดินทางไปร่วมงานศพ ตจว'],
                'high': ['กลับภูมิลำเนา', 'มีธุระที่บ้าน', 'อยู่ต่างจังหวัด', 'เดินทางออกต่างจังหวัด', 'ดูแลพ่อกับแม่ที่ตอนนี้', 'อยู่ตจว'],
                'medium': ['ต่างจังหวัด', 'กลับบ้าน', 'บ้านเกิด', 'จังหวัด', 'ตจว'],
                'low': ['บ้านญาติ', 'อีสาน', 'เหนือ', 'ใต้']
            },
            
            'ดูแลครอบครัว': {
                'critical': ['ลูกป่วย', 'พาไปหาหมอ', 'ดูแลผู้ป่วย', 'ญาติเสีย', 'งานศพ', 'คุณปู่เสีย', 'ยายเสีย', 'น้องเสียชีวืต', 'แม่สามีป่วยหนักไม่มีคนดูแล', 'ไปเยี่ยมคุณแม่ที่มีอาการป่วย', 'พาแม่ไปหาหมอ', 'พาคุณแม่ไปโรงพยาบาล', 'ต้องพาแฟนไปพบหมอ', 'ลูกสาวสอบ', 'ไม่มีคนดูลูก', 'ขนของย้ายบ้าน'],
                'high': ['ดูแลผู้สูงอายุ', 'เฝาไข้', 'พาลูกไปสอบ', 'ต้องไปรับ', 'ไม่มีคนดู'],
                'medium': ['ดูแล', 'ช่วย', 'ผู้สูงอายุ', 'ต้องดูแล', 'ครอบครัว', 'ญาติ'],
                'low': ['พี่น้อง', 'สามี', 'ภรรยา', 'โรงเรียน', 'ลูก', 'พ่อ']
            },
            
            'อุปกรณ์ไม่พร้อม': {
                'critical': ['อุปกรณ์เสีย', 'เครื่องเสีย', 'เครื่องดูดฝุ่นเสีย', 'เครื่องมือเสีย', 'ต้องการแม่บ้านที่มี้ครื่องดูดไรฝุ่น', 'มีแต่เครื่องดูดฝุ่นธรรมดา'],
                'high': ['อุปกรณ์ไม่ครบ', 'โทรศัพท์เสีย', 'เครื่องดูดไรฝุ่น', 'อุปกรณ์ไม่พร้อม', 'อุปกรณ์ไม่ครบ', 'เครื่องดูดฝุ่น'],
                'medium': ['อุปกรณ์', 'เครื่องมือ', 'ชำรุด', 'พัง', 'เสีย'],
                'low': ['หาย', 'ลืม', 'ซ่อม']
            },
            
            'แจ้งลาออก': {
                'critical': ['ลาออก', 'แจ้งลาออก', 'ลาออกจากงาน', 'ขอออกจากงานก่อน', 'ต้องออกจากงานทางครอบครัว'],
                'high': ['ออกจากงาน', 'เลิกงาน', 'ต้องการลาออก', 'จะลาออก'],
                'medium': ['หยุดงาน', 'เปลี่ยนงาน', 'มีงานใหม่'],
                'low': ['สิ้นสุดสัญญา', 'หมดสัญญา']
            },
            
            'อุบัติเหตุ': {
                'critical': ['อุบัติเหตุ', 'ประสบอุบัติเหตุ', 'เกิดอุบัติเหตุ', 'หกล้มข้อเท้าพลิก'],
                'high': ['รถชน', 'รถล้ม', 'บาดเจ็บจากรถล้ม', 'โดนรถชน', 'ขาหัก', 'หกล้ม', 'โดนสุนัขกัด'],
                'medium': ['รถคว่ำ', 'บาดเจ็บจากอุบัติเหตุ'],
                'low': ['ชนกัน', 'ฟกช้ำ', 'เสียหลัก']
            },
            
            'ที่อยู่ไม่ตรงในใบงาน': {
                'critical': ['ที่อยู่ไม่ตรง', 'ที่อยู่ผิด', 'หาไม่เจอบ้าน', 'ใบงานผิด', 'ก่อนกดรับงานเห็นเป็นเขตวัฒนาแต่พอกดรับแล้วไม่ใช่เขตวัฒนา', 'ดูเส้นทางผิด'],
                'high': ['หลงทาง', 'สถานที่ไม่เจอ', 'หมุดไม่ตรง'],
                'medium': ['ที่อยู่', 'สถานที่', 'หาไม่เจอ', 'ข้อมูลผิด'],
                'low': ['ตำแหน่ง', 'อาคารผิด', 'ห้องผิด']
            },
            
            'สัตว์เลี้ยง': {
                'critical': ['สัตว์เลี้ยง', 'กลัวหมา', 'กลัวแมว', 'แพ้ขนสัตว์', 'แพ้ขนแมว'],
                'high': ['แพ้ขน', 'กลัวสัตว์'],
                'medium': ['สัตว์', 'ดุ', 'หมา', 'แมว'],
                'low': ['นก', 'ปลา', 'เห่า']
            }
        }
        
        # "อื่นๆ" category keywords
        self.other_keywords = [
            'ไม่แจ้งเหตุผล', 'ไม่ได้บอกสาเหตุ', 'หาแม่บ้านอื่น', 'หาคนอื่น', 'สอบถามเหตุผลเพิ่มเติม', 'ไม่ได้แจ้งค่ะ'
        ]
        
        # Special compound logic for handling multiple reasons
        self.compound_rules = [
            # If mentions both illness and travel, prioritize travel
            {
                'conditions': ['ป่วย', 'กลับ.*ต่างจังหวัด|ต่างจังหวัด|ตจว'],
                'result': 'กลับต่างจังหวัด',
                'confidence_boost': 0.3
            },
            # If mentions family member illness/death, it's family care
            {
                'conditions': ['แม่.*ป่วย|พ่อ.*ป่วย|ลูก.*ป่วย|ญาติ.*เสีย|ปู่.*เสีย|ยาย.*เสีย|พา.*หมอ'],
                'result': 'ดูแลครอบครัว',
                'confidence_boost': 0.2
            },
            # If mentions customer postponing frequently, it's service conditions
            {
                'conditions': ['ลูกค้า.*เลื่อน.*บ่อย|เลื่อน.*กะทันหัน.*เสียโอกาส'],
                'result': 'ไม่สะดวกเข้าให้บริการ(เงื่อนไขคุณลูกค้า)',
                'confidence_boost': 0.3
            }
        ]
        
        # Initialize ThaiNLP components if available
        if THAINLP_AVAILABLE:
            try:
                self.stopwords = thai_stopwords()
                print("✅ ThaiNLP components initialized")
            except:
                self.stopwords = set()
                print("⚠️ Using basic mode")
        else:
            self.stopwords = set()
    
    def preprocess_text(self, text):
        """Preprocess Thai text with or without ThaiNLP"""
        if pd.isna(text) or text is None:
            return ""
        
        text = str(text).strip()
        
        # Manual corrections for common patterns
        corrections = {
            'เเ': 'แ',
            'คุณเเม่บ้าน': 'แม่บ้าน',
            'คุณแม่บ้าน': 'แม่บ้าน',
            'คุณลูกค้า': 'ลูกค้า',
            'ปจ้ง': 'แจ้ง',
            'คุรแม่บ้าน': 'แม่บ้าน',
            'คุณแม่ย้าน': 'แม่บ้าน',
            'ตั่งครรภ์': 'ตั้งครรภ์',
            'เสียชีวืต': 'เสียชีวิต',
            'วันที่นัดหมาย' : 'วันที่'
        }
        
        for wrong, correct in corrections.items():
            text = text.replace(wrong, correct)
        
        if THAINLP_AVAILABLE:
            try:
                # Use ThaiNLP processing
                text = normalize(text)
                tokens = word_tokenize(text, engine='newmm')
                
                # Clean tokens but keep important ones
                cleaned_tokens = []
                for token in tokens:
                    token = token.strip().lower()
                    if (len(token) > 0 and 
                        not re.match(r'^[0-9\s\W]+$', token)):
                        cleaned_tokens.append(token)
                
                return ' '.join(cleaned_tokens)
            except:
                # Fallback to basic processing
                pass
        
        # Basic processing
        text = text.lower()
        text = re.sub(r'#\w+', '', text)  # Remove hashtags
        text = re.sub(r'\d+', '', text)   # Remove numbers  
        text = re.sub(r'[^\u0E00-\u0E7F\s]', ' ', text)  # Keep Thai and spaces
        text = re.sub(r'\s+', ' ', text)  # Normalize spaces
        
        return text.strip()
    
    def check_compound_rules(self, text):
        """Check for compound logic rules"""
        for rule in self.compound_rules:
            conditions_met = all(
                re.search(condition, text) for condition in rule['conditions']
            )
            if conditions_met:
                return rule['result'], rule['confidence_boost']
        return None, 0
    
    def calculate_score(self, text, category_keywords):
        """Calculate weighted score with improved logic"""
        score = 0
        matched_keywords = []
        
        # NEW: Weight system with higher critical weight
        weights = {'critical': 10, 'high': 5, 'medium': 2, 'low': 1}
        
        for weight_level, keywords in category_keywords.items():
            weight = weights[weight_level]
            
            for keyword in keywords:
                if keyword in text:
                    score += weight
                    matched_keywords.append(keyword)
        
        return score, matched_keywords
    
    def classify_text(self, text):
        """Classify a single text with compound logic"""
        processed_text = self.preprocess_text(text)
        
        if not processed_text.strip():
            return {
                'text': text,
                'predicted_category': 'อื่นๆ',
                'confidence': 0.15,
                'matched_keywords': [],
                'keyword_count': 0,
                'weighted_score': 0,
                'processed_text': processed_text,
                'method': 'empty'
            }
        
        # Check compound rules first
        compound_category, compound_boost = self.check_compound_rules(processed_text)
        
        # Check for "อื่นๆ" keywords
        other_matches = [kw for kw in self.other_keywords if kw in processed_text]
        if other_matches and not compound_category:
            return {
                'text': text,
                'predicted_category': 'อื่นๆ',
                'confidence': 0.8,
                'matched_keywords': other_matches,
                'keyword_count': len(other_matches),
                'weighted_score': len(other_matches) * 3,
                'processed_text': processed_text,
                'method': 'explicit_other'
            }
        
        # Calculate scores for each category
        category_scores = {}
        all_matches = {}
        category_details = {}
        
        for category, keywords_by_weight in self.keyword_patterns.items():
            score, matches = self.calculate_score(processed_text, keywords_by_weight)
            
            if score > 0:
                category_scores[category] = score
                all_matches[category] = matches
                
                # Count critical matches for tie-breaking
                critical_count = sum(1 for keyword in keywords_by_weight.get('critical', []) if keyword in processed_text)
                high_count = sum(1 for keyword in keywords_by_weight.get('high', []) if keyword in processed_text)
                
                category_details[category] = {
                    'score': score,
                    'critical_matches': critical_count,
                    'high_matches': high_count,
                    'total_matches': len(matches)
                }
        
        # Apply compound rule if found
        if compound_category and compound_category in category_scores:
            category_scores[compound_category] += 5  # Boost compound rule result
        
        # Determine best category
        if category_scores:
            # Priority 1: Compound rules
            if compound_category and compound_category in category_scores:
                best_category = compound_category
                method = 'compound_rule'
            else:
                # Priority 2: Categories with critical matches
                categories_with_critical = [cat for cat, details in category_details.items() 
                                           if details['critical_matches'] > 0]
                
                if categories_with_critical:
                    # Among critical matches, pick highest score
                    best_category = max(categories_with_critical, key=lambda cat: (
                        category_details[cat]['critical_matches'],
                        category_details[cat]['score']
                    ))
                    method = 'critical_match'
                else:
                    # Priority 3: Highest score with tie-breaker
                    max_score = max(category_scores.values())
                    top_categories = [cat for cat, score in category_scores.items() if score == max_score]
                    
                    if len(top_categories) == 1:
                        best_category = top_categories[0]
                    else:
                        # Tie-breaker: most high-weight matches
                        best_category = max(top_categories, 
                                           key=lambda cat: category_details[cat]['high_matches'])
                    method = 'highest_score'
            
            best_score = category_scores[best_category]
            matched_keywords = all_matches[best_category]
            
            # Calculate confidence
            text_length = len(processed_text.split())
            keyword_count = len(matched_keywords)
            
            # Base confidence from score
            max_possible_score = 30  # 3 critical keywords * 10 points each
            base_confidence = min(best_score / max_possible_score, 0.6)
            
            # Boosts
            critical_boost = category_details[best_category]['critical_matches'] * 0.2
            compound_boost_applied = compound_boost if best_category == compound_category else 0
            thainlp_boost = 0.1 if THAINLP_AVAILABLE else 0
            keyword_boost = min(keyword_count * 0.05, 0.2)
            
            confidence = base_confidence + critical_boost + compound_boost_applied + thainlp_boost + keyword_boost
            confidence = min(max(confidence, 0.3), 1.0)
            
        else:
            best_category = 'อื่นๆ'
            confidence = 0.25
            matched_keywords = []
            best_score = 0
            keyword_count = 0
            method = 'no_match'
        
        return {
            'text': text,
            'predicted_category': best_category,
            'confidence': round(confidence, 3),
            'matched_keywords': matched_keywords,
            'keyword_count': len(matched_keywords),
            'weighted_score': best_score,
            'processed_text': processed_text,
            'method': method
        }
    
    def classify_dataframe(self, df, text_column='remark'):
        """Classify texts in a DataFrame"""
        enhancement_info = "with ThaiNLP" if THAINLP_AVAILABLE else "basic mode"
        print(f"🚀 Processing {len(df)} records ({enhancement_info})...")
        
        if text_column not in df.columns:
            raise ValueError(f"Column '{text_column}' not found in DataFrame")
        
        # Classify each text
        results = []
        for idx, row in df.iterrows():
            text = str(row[text_column]) if not pd.isna(row[text_column]) else ""
            result = self.classify_text(text)
            results.append(result)
            
            if (idx + 1) % 500 == 0:
                print(f"  ✓ Processed {idx + 1}/{len(df)} records...")
        
        # Create output DataFrame
        output_df = df.copy()
        
        # Add prediction columns
        output_df['category'] = [r['predicted_category'] for r in results]
        output_df['confidence'] = [r['confidence'] for r in results]
        output_df['Keywords'] = [', '.join(r['matched_keywords'][:5]) if r['matched_keywords'] else '' for r in results]
        output_df['keyword_count'] = [r['keyword_count'] for r in results]
        output_df['weighted_score'] = [r['weighted_score'] for r in results]
        output_df['processed_text'] = [r['processed_text'] for r in results]
        output_df['method'] = [r['method'] for r in results]
        
        return output_df, results

def apply_fixed_v3(input_data, output_file=None):
    """Apply the FIXED classifier V3 to your data"""
    print("🛠️  APPLYING FIXED THAI CLASSIFIER V3")
    print("=" * 50)
    
    # Load data
    if isinstance(input_data, pd.DataFrame):
        df = input_data
        print(f"✓ Using provided DataFrame with {len(df)} records")
    else:
        try:
            if str(input_data).endswith('.xlsx'):
                df = pd.read_excel(input_data)
            else:
                df = pd.read_csv(input_data)
            print(f"✓ Loaded {len(df)} records from file")
        except Exception as e:
            print(f"❌ Error loading data: {str(e)}")
            return None
    
    # Apply FIXED classifier V3
    classifier = FixedThaiClassifierV3()
    fixed_df, results = classifier.classify_dataframe(df, 'remark')
    
    # Show results
    print(f"\n📈 FIXED V3 CLASSIFICATION RESULTS:")
    avg_conf = fixed_df['confidence'].mean()
    high_conf = (fixed_df['confidence'] >= 0.7).sum()
    medium_conf = ((fixed_df['confidence'] >= 0.4) & (fixed_df['confidence'] < 0.7)).sum()
    low_conf = (fixed_df['confidence'] < 0.4).sum()
    
    print(f"Average confidence: {avg_conf:.3f}")
    print(f"High confidence (≥0.7): {high_conf} ({high_conf/len(fixed_df)*100:.1f}%)")
    print(f"Medium confidence (0.4-0.7): {medium_conf} ({medium_conf/len(fixed_df)*100:.1f}%)")
    print(f"Low confidence (<0.4): {low_conf} ({low_conf/len(fixed_df)*100:.1f}%)")
    
    # Show method breakdown
    method_counts = fixed_df['method'].value_counts()
    print(f"\n🔍 Classification methods:")
    for method, count in method_counts.items():
        print(f"  {method}: {count} ({count/len(fixed_df)*100:.1f}%)")
    
    # Save results
    if output_file is not None:
        try:
            fixed_df.to_csv(output_file, index=False, encoding='utf-8-sig')
            print(f"\n💾 Results saved to: {output_file}")
        except Exception as e:
            print(f"❌ Error saving file: {str(e)}")
    
    return fixed_df

# Test the FIXED V3 classifier with the problematic cases
print("\n" + "="*60)
print("🧪 TESTING FIXED V3 CLASSIFIER ON PROBLEMATIC CASES")
print("="*60)

# Create FIXED V3 classifier instance
classifier = FixedThaiClassifierV3()

# Test the exact problematic cases from your data
problematic_cases = [
    "แม่ป่วย ต้องกลับบ้านต่างจังหวัดด่วนเลยคะ",  # Should be กลับต่างจังหวัด
    "แม่บ้านตั่งครรภ์ เริ่มทำไม่ไหว้แล้วค่ะ",  # Should be ปัญหาสุขภาพ
    "ข้อเท้าอักเสบค่ะ",  # Should be ป่วย
    "สุขภาพแม่บ้านไม่ค่อยดีช่วงนี้ค่ะ",  # Should be ปัญหาสุขภาพ
    "ลูกค้ามีการเลื่อนการจองอยู่บ่อยครั้ง และมีการเลื่อนกะทันหัน ทำให้แม่บ้านเสียโอกาสในการรับงานอื่น",  # Should be ไม่สะดวกเข้าให้บริการ(เงื่อนไขคุณลูกค้า)
    "จำเป็นต้องไปเยี่ยมคุณแม่ที่มีอาการป่วย",  # Should be ดูแลครอบครัว
    "พาแม่ไปหาหมอ",  # Should be ดูแลครอบครัว
    "คุณปู่เสีย ค่ะ",  # Should be ดูแลครอบครัว
    "ผ่าฟันคุดค่ะ",  # Should be ป่วย
    "ติดงานขนส่งค่ะ",  # Should be ติดงานประจำ
    "ติดธุระต่างจังหวัด",  # Should be กลับต่างจังหวัด
    "ก่อนกดรับงานเห็นเป็นเขตวัฒนาแต่พอกดรับแล้วไม่ใช่เขตวัฒนา",  # Should be ที่อยู่ไม่ตรงในใบงาน
]

for i, text in enumerate(problematic_cases, 1):
    result = classifier.classify_text(text)
    print(f"\n{i}. Text: {text}")
    print(f"   ✅ Category: {result['predicted_category']}")
    print(f"   📊 Confidence: {result['confidence']}")
    print(f"   🔑 Keywords: {', '.join(result['matched_keywords'][:3])}")
    print(f"   🔧 Method: {result['method']}")

print(f"\n🎉 FIXED V3 Classifier is ready to use!")
print(f"\n📝 Usage:")
print(f"fixed_v3_df = apply_fixed_v3(problem_df)")

⚠️ ThaiNLP not available. Using basic preprocessing.

🧪 TESTING FIXED V3 CLASSIFIER ON PROBLEMATIC CASES
🚀 Initializing FIXED Thai Classifier V3...

1. Text: แม่ป่วย ต้องกลับบ้านต่างจังหวัดด่วนเลยคะ
   ✅ Category: กลับต่างจังหวัด
   📊 Confidence: 0.817
   🔑 Keywords: ต่างจังหวัด, กลับบ้าน, จังหวัด
   🔧 Method: compound_rule

2. Text: แม่บ้านตั่งครรภ์ เริ่มทำไม่ไหว้แล้วค่ะ
   ✅ Category: ปัญหาสุขภาพ
   📊 Confidence: 1.0
   🔑 Keywords: เริ่มทำไม่ไหว้แล้ว, ครรภ์, ตั้งครรภ์
   🔧 Method: critical_match

3. Text: ข้อเท้าอักเสบค่ะ
   ✅ Category: ป่วย
   📊 Confidence: 0.583
   🔑 Keywords: ข้อเท้าอักเสบ
   🔧 Method: critical_match

4. Text: สุขภาพแม่บ้านไม่ค่อยดีช่วงนี้ค่ะ
   ✅ Category: ปัญหาสุขภาพ
   📊 Confidence: 0.3
   🔑 Keywords: สุขภาพ
   🔧 Method: highest_score

5. Text: ลูกค้ามีการเลื่อนการจองอยู่บ่อยครั้ง และมีการเลื่อนกะทันหัน ทำให้แม่บ้านเสียโอกาสในการรับงานอื่น
   ✅ Category: ไม่สะดวกเข้าให้บริการ(เงื่อนไขคุณลูกค้า)
   📊 Confidence: 0.8
   🔑 Keywords: เสียโอกาสในการรับงานอื่น, เลื่อ

In [87]:
fixed_v3_df = apply_fixed_v3(result_dataframe)

fixed_v3_df['category'] = np.where(fixed_v3_df['problem'] == 'คืนงาน (โควิด)', 'โควิด', fixed_v3_df['category'])

fixed_v3_df

🛠️  APPLYING FIXED THAI CLASSIFIER V3
✓ Using provided DataFrame with 2177 records
🚀 Initializing FIXED Thai Classifier V3...
🚀 Processing 2177 records (basic mode)...
  ✓ Processed 500/2177 records...
  ✓ Processed 1000/2177 records...
  ✓ Processed 1500/2177 records...
  ✓ Processed 2000/2177 records...

📈 FIXED V3 CLASSIFICATION RESULTS:
Average confidence: 0.481
High confidence (≥0.7): 455 (20.9%)
Medium confidence (0.4-0.7): 438 (20.1%)
Low confidence (<0.4): 1284 (59.0%)

🔍 Classification methods:
  highest_score: 1262 (58.0%)
  critical_match: 704 (32.3%)
  no_match: 130 (6.0%)
  compound_rule: 62 (2.8%)
  explicit_other: 19 (0.9%)


,report_date,report_month,problem,order_id,report_datetime,day_of_week,selected_professional_id,cancel_professional_id,booking_type,remark,category,confidence,Keywords,keyword_count,weighted_score,processed_text,method
0,2025-01-01,2025-01,คืนงาน (กะทันหัน),1192844,2025-01-01 06:55:00,Wed,0,8246,งานใครรับก็ได้,คุณแม่บ้านแจ้งเป็นไข้ทับระดูและท้องเสีย ลุกไม...,ป่วย,1.000,"ไข้, ท้องเสีย, เป็นไข้",3,25,แม่บ้านแจ้งเป็นไข้ทับระดูและท้องเสีย ลุกไม่ไหว...,critical_match
1,2025-01-01,2025-01,คืนงาน (กะทันหัน),1191526,2025-01-01 11:03:00,Wed,0,10287,งานใครรับก็ได้,คุณแม่บ้านแจ้งคืนงานเนื่องจาก ไม่สบาย มีไข้ น้...,ป่วย,0.967,"ไข้, ไอ, เวียนหัว, ไม่สบาย, ยา",5,17,แม่บ้านแจ้งคืนงานเนื่องจาก ไม่สบาย มีไข้ น้ำมู...,critical_match
2,2025-01-01,2025-01,คืนงาน (ล่วงหน้า),1154597,2025-01-01 10:11:00,Wed,0,79,งานใครรับก็ได้,คุณแม่บ้านแจ้งคืนงานเนื่องจาก แม่บ้านประสบอุบั...,อุบัติเหตุ,1.000,"อุบัติเหตุ, ประสบอุบัติเหตุ",2,20,แม่บ้านแจ้งคืนงานเนื่องจาก แม่บ้านประสบอุบัติเ...,critical_match
3,2025-01-01,2025-01,คืนงาน (กะทันหัน),1018893,2025-01-01 11:50:00,Wed,0,11090,งานใครรับก็ได้,คุณแม่บ้านแจ้งคืนงาน 14 งาน เนื่องจากจะลาออก ต...,แจ้งลาออก,0.800,"ลาออก, จะลาออก",2,15,แม่บ้านแจ้งคืนงาน งาน เนื่องจากจะลาออก ตั้งครร...,critical_match
4,2025-01-01,2025-01,คืนงาน (กะทันหัน),1108624,2025-01-01 14:45:00,Wed,0,7810,งานใครรับก็ได้,คุณแม่บ้านแจ้งมีอาการปวดหัวจึงยังไม่สะดวกเข้าบ...,ป่วย,0.300,"ปวดหัว, อาการ",2,3,แม่บ้านแจ้งมีอาการปวดหัวจึงยังไม่สะดวกเข้าบริการ,highest_score
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2172,2025-06-23,2025-06,คืนงาน (ล่วงหน้า),1373885,2025-06-23 11:59:00,Mon,0,12876,งานใครรับก็ได้,คุณแม่บ้านแจ้งว่ากดรับงานผิดค่ะ,ดูวันผิด,0.300,"กดรับงานผิด, กด",2,6,แม่บ้านแจ้งว่ากดรับงานผิดค่ะ,highest_score
2173,2025-06-23,2025-06,คืนงาน (กะทันหัน),1373787,2025-06-23 23:38:00,Mon,1360,1360,งานจองตรง,รับเคสจาก cc คุณลูกค้าแจ้งว่า คุณแม่บ้านไม่สะด...,ไม่สะดวกเข้าให้บริการ(เงื่อนไขคุณลูกค้า),0.667,"ไม่สะดวกเข้าให้บริการ, เปลี่ยน",2,11,รับเคสจาก ลูกค้าแจ้งว่า แม่บ้านไม่สะดวกเข้าให้...,critical_match
2174,2025-06-23,2025-06,คืนงาน (กะทันหัน),1371647,2025-06-23 12:00:00,Mon,0,10579,งานใครรับก็ได้,คุณลูกค้าแจ้งเข้ามาว่าคุณแม่บ้านไม่สะดวกเดินทา...,ไม่สะดวกเดินทาง,0.383,"ไม่สะดวกเดินทาง, เดินทาง, ไม่สะดวก",3,7,ลูกค้าแจ้งเข้ามาว่าแม่บ้านไม่สะดวกเดินทางเข้าใ...,highest_score
2175,2025-06-23,2025-06,คืนงาน (ล่วงหน้า),1364819,2025-06-23 08:12:00,Mon,10258,10258,งานจองตรง,ต้องเดินทางไปต่างประเทศ,ติดธุระด่วน,0.300,ต่างประเทศ,1,5,ต้องเดินทางไปต่างประเทศ,highest_score


In [78]:
check_cat_other = fixed_v3_df[(fixed_v3_df['category'] == 'อื่นๆ') & (fixed_v3_df['report_month'] == '2025-05')]

check_cat_other

,report_date,report_month,problem,order_id,report_datetime,day_of_week,selected_professional_id,cancel_professional_id,booking_type,remark,category,confidence,Keywords,keyword_count,weighted_score,processed_text,method
1491,2025-05-06,2025-05,คืนงาน (ล่วงหน้า),1318121,2025-05-06 07:18:00,Tue,0,12550,งานใครรับก็ได้,คุณแม่บ้านคืนงาน,อื่นๆ,0.25,,0,0,แม่บ้านคืนงาน,no_match
1502,2025-05-06,2025-05,คืนงาน (ล่วงหน้า),1278497,2025-05-06 13:21:00,Tue,0,9713,งานใครรับก็ได้,คุณแม่บ้านคืนงาน,อื่นๆ,0.25,,0,0,แม่บ้านคืนงาน,no_match
1533,2025-05-08,2025-05,คืนงาน (กะทันหัน),1164620,2025-05-08 19:45:00,Thu,0,2107,งานใครรับก็ได้,แจ้งคืนงานเนื่องจากมไ่สะดวกเข้าให้บริการ,อื่นๆ,0.25,,0,0,แจ้งคืนงานเนื่องจากมไ่สะดวกเข้าให้บริการ,no_match
1535,2025-05-09,2025-05,คืนงาน (ล่วงหน้า),1326143,2025-05-09 08:42:00,Fri,0,12039,งานใครรับก็ได้,คุณแม่บ้านแจ้งคืนงาน ไม่แจ้งเหตุผลเพิ่มเติม,อื่นๆ,0.80,ไม่แจ้งเหตุผล,1,3,แม่บ้านแจ้งคืนงาน ไม่แจ้งเหตุผลเพิ่มเติม,explicit_other
1536,2025-05-09,2025-05,คืนงาน (กะทันหัน),1325608,2025-05-09 09:31:00,Fri,0,10364,งานใครรับก็ได้,คุณแม่บ้านแจ้งว่า เครืองดูดฝุ่นมีปัญหา ต้องการ...,อื่นๆ,0.25,,0,0,แม่บ้านแจ้งว่า เครืองดูดฝุ่นมีปัญหา ต้องการคืน...,no_match
1550,2025-05-10,2025-05,คืนงาน (กะทันหัน),1317436,2025-05-10 07:13:00,Sat,0,10636,งานใครรับก็ได้,คุณแม่บ้านแจ้งคืนงาน ไม่แจ้งเหตุผลเพิ่มเติม\nค...,อื่นๆ,0.80,ไม่แจ้งเหตุผล,1,3,แม่บ้านแจ้งคืนงาน ไม่แจ้งเหตุผลเพิ่มเติม แม่บ้...,explicit_other
1553,2025-05-10,2025-05,คืนงาน (กะทันหัน),1298037,2025-05-10 08:05:00,Sat,0,12172,งานใครรับก็ได้,คุณแม่บ้านแจ้งคืนงานเนื่องจากจะกลับไปเลือกตั้ง,อื่นๆ,0.25,,0,0,แม่บ้านแจ้งคืนงานเนื่องจากจะกลับไปเลือกตั้ง,no_match
1594,2025-05-13,2025-05,คืนงาน (ล่วงหน้า),1297512,2025-05-13 07:33:00,Tue,0,11986,งานใครรับก็ได้,คุณแม่บ้านแจ้งคืนงาน,อื่นๆ,0.25,,0,0,แม่บ้านแจ้งคืนงาน,no_match
1665,2025-05-18,2025-05,คืนงาน (กะทันหัน),1250473,2025-05-18 10:31:00,Sun,0,12039,งานใครรับก็ได้,คุณแม่บ้านขอคืนงาน ไม่แจ้งเหตุผล,อื่นๆ,0.80,ไม่แจ้งเหตุผล,1,3,แม่บ้านขอคืนงาน ไม่แจ้งเหตุผล,explicit_other
1669,2025-05-18,2025-05,คืนงาน (กะทันหัน),1335028,2025-05-18 15:08:00,Sun,0,12575,งานใครรับก็ได้,คุณแม่บ้านแจ้งคืนงานเนื่องจาก ไม่แจ้งเหตุผลเพิ...,อื่นๆ,0.80,ไม่แจ้งเหตุผล,1,3,แม่บ้านแจ้งคืนงานเนื่องจาก ไม่แจ้งเหตุผลเพิ่มเติม,explicit_other


In [89]:
check_cat_other = fixed_v3_df[(fixed_v3_df['category'] == 'ไม่สะดวกเดินทาง') & (fixed_v3_df['report_month'] == '2025-05')]

check_cat_other

,report_date,report_month,problem,order_id,report_datetime,day_of_week,selected_professional_id,cancel_professional_id,booking_type,remark,category,confidence,Keywords,keyword_count,weighted_score,processed_text,method
1434,2025-05-01,2025-05,คืนงาน (กะทันหัน),1317744,2025-05-01 17:50:00,Thu,0,12195,งานใครรับก็ได้,คุณแม่บ้านแจ้งขอคืนงาน เนื่องจาก กดผิดค่ะลืมดู...,ไม่สะดวกเดินทาง,0.300,"ไปไม่ทัน, ไม่สะดวก",2,6,แม่บ้านแจ้งขอคืนงาน เนื่องจาก กดผิดค่ะลืมดูเวล...,highest_score
1438,2025-05-02,2025-05,คืนงาน (กะทันหัน),1317570,2025-05-02 08:21:00,Fri,0,7125,งานใครรับก็ได้,คุณแม่บ้านคืนงาน แจ้งถึงหน้างาน แต่นิติให้เข้...,ไม่สะดวกเดินทาง,0.300,ไม่สะดวก,1,1,แม่บ้านคืนงาน แจ้งถึงหน้างาน แต่นิติให้เข้าบริ...,highest_score
1447,2025-05-02,2025-05,คืนงาน (กะทันหัน),1318682,2025-05-02 14:09:00,Fri,0,12373,งานใครรับก็ได้,คุณแม่บ้านแจ้งคืนงานเนื่องจากไปไม่ทัน,ไม่สะดวกเดินทาง,0.300,ไปไม่ทัน,1,5,แม่บ้านแจ้งคืนงานเนื่องจากไปไม่ทัน,highest_score
1449,2025-05-02,2025-05,คืนงาน (โครงการป้อนงานคุณแม่บ้าน),1318386,2025-05-02 15:09:00,Fri,0,12096,งาน Auto-Assign,คุณแม่บ้านแจ้งคืนงานเนื่องจากไม่สะดวกเดินทาง,ไม่สะดวกเดินทาง,0.383,"ไม่สะดวกเดินทาง, เดินทาง, ไม่สะดวก",3,7,แม่บ้านแจ้งคืนงานเนื่องจากไม่สะดวกเดินทาง,highest_score
1450,2025-05-02,2025-05,คืนงาน (โครงการป้อนงานคุณแม่บ้าน),1317410,2025-05-02 16:07:00,Fri,0,8412,งาน Auto-Assign,คุณลูกค้าต้องการคุณแม่บ้านท่านที่สามารถสื่อสาร...,ไม่สะดวกเดินทาง,0.300,รถ,1,1,ลูกค้าต้องการแม่บ้านท่านที่สามารถสื่อสารภาษาได...,highest_score
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830,2025-05-30,2025-05,คืนงาน (กะทันหัน),1345939,2025-05-30 15:14:00,Fri,0,6455,งานใครรับก็ได้,แม่บ้านคืนงานเนื่องจากรถเสีย,ไม่สะดวกเดินทาง,0.667,"รถเสีย, รถ",2,11,แม่บ้านคืนงานเนื่องจากรถเสีย,critical_match
1835,2025-05-30,2025-05,คืนงาน (กะทันหัน),1312666,2025-05-30 18:53:00,Fri,0,6998,งานใครรับก็ได้,คุณแม่บ้านขอคืนงานเนื่องจากไม่สะดวกในการเดินทา...,ไม่สะดวกเดินทาง,0.300,"เดินทาง, ไม่สะดวก",2,2,แม่บ้านขอคืนงานเนื่องจากไม่สะดวกในการเดินทางค่ะ,highest_score
1838,2025-05-31,2025-05,คืนงาน (กะทันหัน),1343676,2025-05-31 09:19:00,Sat,0,12700,งานใครรับก็ได้,คุณแม่บ้านแจ้งไม่สามารถเข้าให้บริการได้เนื่องจ...,ไม่สะดวกเดินทาง,0.300,รถ,1,1,แม่บ้านแจ้งไม่สามารถเข้าให้บริการได้เนื่องจากไ...,highest_score
1841,2025-05-31,2025-05,คืนงาน (ล่วงหน้า),1349274,2025-05-31 08:01:00,Sat,0,6416,งานใครรับก็ได้,มันไกลด้วยคะนั่งรถ4ต่อ เดินทางไม่สะดวกคะ,ไม่สะดวกเดินทาง,0.300,"เดินทาง, รถ, ไม่สะดวก",3,3,มันไกลด้วยคะนั่งรถต่อ เดินทางไม่สะดวกคะ,highest_score
